In [5]:

%%writefile orderbook.cpp
/*This project implements a miniature stock exchange order-book matching engine in C++,
showcasing practical Data Structures and Algorithms used in high-frequency trading.
It maintains buy and sell orders using priority queues with custom comparators,
applies strict price–time priority, and executes trades whenever compatible orders match*/

#include <bits/stdc++.h>
using namespace std;

struct Order {
    int id;
    int quantity;
    double price;
    long long timestamp;
    bool isBuy;
};

// Max-heap for BUY (highest price first)
struct BuyCmp {
    bool operator()(const Order &a, const Order &b) const {
        if (a.price == b.price) return a.timestamp > b.timestamp;
        return a.price < b.price;
    }
};

// Min-heap for SELL (lowest price first)
struct SellCmp {
    bool operator()(const Order &a, const Order &b) const {
        if (a.price == b.price) return a.timestamp > b.timestamp;
        return a.price > b.price;
    }
};

class OrderBook {
    priority_queue<Order, vector<Order>, BuyCmp> buyHeap;
    priority_queue<Order, vector<Order>, SellCmp> sellHeap;
    long long ts = 0;

public:
    void addBuy(int id, int qty, double price) {
        matchOrder({id, qty, price, ts++, true});
    }

    void addSell(int id, int qty, double price) {
        matchOrder({id, qty, price, ts++, false});
    }

private:
    void matchOrder(Order incoming) {
        if (incoming.isBuy)
            matchBuy(incoming);
        else
            matchSell(incoming);
    }

    void matchBuy(Order buy) {
        while (!sellHeap.empty() && buy.quantity > 0 && sellHeap.top().price <= buy.price) {
            Order sell = sellHeap.top(); sellHeap.pop();
            int traded = min(buy.quantity, sell.quantity);

            cout << "TRADE: " << traded
                 << " @ " << sell.price
                 << " (BUY " << buy.id << " vs SELL " << sell.id << ")\n";

            buy.quantity -= traded;
            sell.quantity -= traded;

            if (sell.quantity > 0) sellHeap.push(sell);
        }
        if (buy.quantity > 0) buyHeap.push(buy);
    }

    void matchSell(Order sell) {
        while (!buyHeap.empty() && sell.quantity > 0 && buyHeap.top().price >= sell.price) {
            Order buy = buyHeap.top(); buyHeap.pop();
            int traded = min(sell.quantity, buy.quantity);

            cout << "TRADE: " << traded
                 << " @ " << buy.price
                 << " (SELL " << sell.id << " vs BUY " << buy.id << ")\n";

            sell.quantity -= traded;
            buy.quantity -= traded;

            if (buy.quantity > 0) buyHeap.push(buy);
        }
        if (sell.quantity > 0) sellHeap.push(sell);
    }
};

int main() {
    OrderBook ob;
    ob.addBuy(1, 100, 50);
    ob.addSell(2, 50, 48);
    ob.addSell(3, 70, 50);
    ob.addBuy(4, 30, 52);
    return 0;
}


Writing orderbook.cpp


In [6]:
!g++ orderbook.cpp -o orderbook


In [7]:
!./orderbook


TRADE: 50 @ 50 (SELL 2 vs BUY 1)
TRADE: 50 @ 50 (SELL 3 vs BUY 1)
TRADE: 20 @ 50 (BUY 4 vs SELL 3)
